In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///mta_data.db')

In [1]:
df_july = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '07/%/2020';", engine)
df_july.head()

In [ ]:
df_aug = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '08/%/2020';", engine)
df_aug.head()

In [ ]:
df_sep = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '09/%/2020';", engine)
df_sep.head()

In [ ]:
df_oct = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '10/%/2020';", engine)
df_oct.head()

In [ ]:
df_nov = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '11/%/2020';", engine)
df_nov.head()

In [ ]:
df_dec = pd.read_sql("SELECT * from mta_data WHERE DATE LIKE '12/%/2020';", engine)
df_dec.head()

In [ ]:
df = pd.concat([df_july, df_aug, df_sep, df_oct, df_nov, df_dec], ignore_index=True)
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

## Cleaning the data

In [ ]:
df.columns = [col.strip() for col in df.columns]
df.columns

In [ ]:
# check and delete rows with null values if any
df.loc[df.isna().sum()]

In [ ]:
# delete rows with values zero
df.loc[df['ENTRIES']==0]

In [ ]:
df.drop(df.loc[df['ENTRIES']==0].index, inplace=True)

Check if rows with zero value are deleted from the dataframe

In [ ]:
df.loc[df['ENTRIES']==0]

The date and time datatypes are string, let's convert date to datetime and form a new column by combining date and time.

In [ ]:
import datetime
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [ ]:
# convert date to datetime type
df["DATE"] = pd.to_datetime(df.DATE)

In [ ]:
df.info()

## Delete duplicate rows

In [ ]:
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

In [ ]:
# delete duplicate rows
df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=False)
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

Check if duplicate values have been deleted

In [ ]:
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

Check the column DESC and the values.

In [ ]:
df.DESC.value_counts()

In [ ]:
df = df.drop(["DESC"], axis=1, errors="ignore")

Each combination of C/A, UNIT, SCP, and STATION represents a unique turnstile. Let's take a look at one specific turnstile on a specific date. What does each row in the dataframe represent? Obtain the maximum ENTRIES value for each day, for each unique turnstile.

In [ ]:
df[(df["C/A"] == "A002") & 
(df["UNIT"] == "R051") & 
(df["SCP"] == "02-00-00") & 
(df["STATION"] == "59 ST") &
(df["DATE"] == "05/25/2020")]

In [ ]:
df_daily = df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False).ENTRIES.max()

In [ ]:
df_daily.head()

Calculate the daily entries i.e the number of people traveling on each day. Group the data by turnstile and use the Pandas .apply() method to compute the same differencing function for each turnstile.

In [ ]:
df_daily[["PREV_DATE", "PREV_ENTRIES"]] = (df_daily.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"].apply(lambda grp: grp.shift(1)))

In [ ]:
df_daily.head()

In [ ]:
df_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

## Reverse Entries
Let's look at some more weirdness in the data and think about handling it before we finalize our daily count column

In [ ]:
df_daily[df_daily["ENTRIES"] < df_daily["PREV_ENTRIES"]].head()

Let's check for one particular turnstyle, the reverse counter

In [ ]:
mask = ((df["C/A"] == "A011") & 
(df["UNIT"] == "R080") & 
(df["SCP"] == "01-00-00") & 
(df["STATION"] == "57 ST-7 AV") &
(df["DATE_TIME"].dt.date == datetime.datetime(2020, 8, 27).date()))

df[mask].head()

Let's see how many stations have this problem

In [ ]:
(df_daily[df_daily["ENTRIES"] < df_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
df_daily["DAILY_ENTRIES"] = df_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
df_daily.head()

So far we've been operating on a single turnstile level. Now let's combine turnstiles that fall within the same ControlArea/Unit/Station combo. There are some ControlArea/Unit/Station groups that have a single turnstile, but most have multiple turnstiles -- same value for the C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together. For each ControlArea/UNIT/STATION combo, for each day, sum the counts from each turnstile belonging to that combo.

In [ ]:
ca_unit_station_daily = df_daily.groupby(["C/A", "UNIT", "STATION", "DATE"])[['DAILY_ENTRIES']].sum().reset_index()
ca_unit_station_daily.head()

In [ ]:
ca_unit_station_daily_sum = ca_unit_station_daily.groupby("DATE")["DAILY_ENTRIES"].mean().reset_index()
ca_unit_station_daily_sum

Below is the daily traffic for the months of May, June, July, August, for the year 2020. We can see the daily traffic is increasing slowly as compared to the beginning of May, 2020.

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(ca_unit_station_daily_sum.DATE, ca_unit_station_daily_sum.DAILY_ENTRIES, color='red', linewidth=3)

We can see the holiday seasons are busiest days from the six month period.

## Calculate the daily entries for the month of July 2020

In [ ]:
df_july_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 7) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_july_2020_daily

In [ ]:
station_daily_july_2020 = df_july_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_july_2020
# manhattan

In [ ]:
sorted(station_daily_july_2020.STATION.unique())

## Calculate the daily entries for the month of August 2020

In [ ]:
df_aug_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 8) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_aug_2020_daily

In [ ]:
station_daily_aug_2020 = df_aug_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_aug_2020
# manhattan financial district

## Calculate the daily entries for the month of September 2020

In [ ]:
df_sep_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 9) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_sep_2020_daily

In [ ]:
station_daily_sep_2020 = df_sep_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_sep_2020

# Queens 

In [ ]:
x = ['34 ST-PENN STA', 'FULTON ST', '34 ST-HERALD SQ', 'GRD CNTRL-42 ST', 'FLUSHING-MAIN', '23 ST', '86 ST', '125 ST']
y = []

plt.figure(figsize=(14, 7))
plt.barh(x, y, color=['grey', 'lightblue', 'pink'])

## Calculate the daily entries for the month of October 2020

In [ ]:
df_oct_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 10) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_oct_2020_daily

In [ ]:
station_daily_oct_2020 = df_oct_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_oct_2020
# manhattan

## Calculate the daily entries for the month of November 2020

In [ ]:
df_nov_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 11) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_nov_2020_daily

In [ ]:
station_daily_nov_2020 = df_nov_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_nov_2020
# manhattan 

## Calculate the daily entries for the month of December 2020

In [ ]:
df_dec_2020_daily = ca_unit_station_daily.loc[(ca_unit_station_daily['DATE'].dt.month == 12) & (ca_unit_station_daily["DATE"].dt.year==2020)][["STATION", "DATE", "DAILY_ENTRIES"]]
df_dec_2020_daily

In [ ]:
station_daily_dec_2020 = df_dec_2020_daily.groupby(["STATION"])[['DAILY_ENTRIES']].sum().reset_index().sort_values('DAILY_ENTRIES', ascending=False)
station_daily_dec_2020
# manhattan

## Calculate the daily time series for each STATION for all the months, by adding up all the turnstiles in a station.

In [ ]:
station_daily = df_daily.groupby(["STATION", "DATE"])[['DAILY_ENTRIES']].sum().reset_index()
station_daily.head()

Sum total ridership for each station and sort them, so you can find out the stations with the highest traffic during the time you investigate.

In [ ]:
station_totals = station_daily.groupby('STATION').sum()\
    .sort_values('DAILY_ENTRIES', ascending=False)\
    .reset_index()

station_totals.head(20)

In [ ]:
x = ['34 ST-PENN STA', 'FULTON ST', '34 ST-HERALD SQ', 'GRD CNTRL-42 ST', 'FLUSHING-MAIN', '23 ST', '86 ST', '125 ST']
y = [7473555.0, 5664641.0, 4923146.0, 4710694.0, 4493024.0, 4412771.0, 4155236.0, 3863366.0]

plt.figure(figsize=(14, 7))
plt.barh(x, y, color=['grey', 'lightblue', 'pink'])

Above are the top 20 stations with the highest traffic. The top 8 stations traffic over the six months, post covid lockdown is shown in the bar plot.